### Diatom Analysis Pipeline - OTU

#### A notebook  for the diatom analysis pipeline using OTU classification.

### 1. Create BLAST reference database

In [1]:
%%time
!makeblastdb -in diatoms.sequences.FINAL2017.fasta -out diatoms -dbtype nucl



Building a new DB, current time: 06/03/2019 09:21:25
New DB name:   /code/diatoms
New DB title:  diatoms.sequences.FINAL2017.fasta
Sequence type: Nucleotide
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 2701 sequences in 0.207415 seconds.
CPU times: user 0 ns, sys: 20 ms, total: 20 ms
Wall time: 1.1 s


### 2. Quality Control
* *Cutadapt:* Trim primers from sequences
* *SicklePE:* Trim off bad quality 3' bases
* *Pear:* Merge R1 and R2 reads
* *SickleSE:* Remove post-merging bad quality sequences
* *Histogram Generation:* Plots sequence length against number of sequences
* *QIIME Prep:* Pepare passed QC files for QIIME processing

In [ ]:
%%time
!python ./ampliconQC.py --data sequences --forward ATGCGTTGGAGAGARCGTTTC --reverse GATCACCTTCTAATTTACCWACAACTG --threads 8 --histograms --qiime

This is cutadapt 1.9.1 with Python 2.7.16
Command line parameters: -e 0.047619047619 -b ATGCGTTGGAGAGARCGTTTC -b GATCACCTTCTAATTTACCWACAACTG -o /code/sequences/1040.R1.fastq.gz.trimmed.fastq.gz /code/sequences/1040.R1.fastq.gz
Trimming 2 adapters with at most 4.8% errors in single-end mode ...
Finished in 3.64 s (38 us/read; 1.59 M reads/minute).

=== Summary ===

Total reads processed:                  96,518
Reads with adapters:                    90,759 (94.0%)
Reads written (passing filters):        96,518 (100.0%)

Total basepairs processed:    23,184,439 bp
Total written (filtered):     21,576,862 bp (93.1%)

=== Adapter 1 ===

Sequence: ATGCGTTGGAGAGARCGTTTC; Type: variable 5'/3'; Length: 21; Trimmed: 73209 times.
73157 times, it overlapped the 5' end of a read
52 times, it overlapped the 3' end or was within the read

No. of allowed errors:
0-21 bp: 0

Overview of removed sequences (5')
length	count	expect	max.err	error counts
3	1	1508.1	0	1
4	1	377.0	0	1
16	2	0.0	0	2
17	1	0.0	

length	count	expect	max.err	error counts
3	41	1467.5	0	41
27	5	0.0	1	3 2


Overview of removed sequences (3' or within)
length	count	expect	max.err	error counts
3	53	1467.5	0	53
4	21808	366.9	0	21808

This is cutadapt 1.9.1 with Python 2.7.16
Command line parameters: -e 0.047619047619 -b ATGCGTTGGAGAGARCGTTTC -b GATCACCTTCTAATTTACCWACAACTG -o /code/sequences/1116.R1.fastq.gz.trimmed.fastq.gz /code/sequences/1116.R1.fastq.gz
Trimming 2 adapters with at most 4.8% errors in single-end mode ...
Finished in 4.84 s (59 us/read; 1.02 M reads/minute).

=== Summary ===

Total reads processed:                  82,309
Reads with adapters:                    76,552 (93.0%)
Reads written (passing filters):        82,309 (100.0%)

Total basepairs processed:    19,856,185 bp
Total written (filtered):     18,480,172 bp (93.1%)

=== Adapter 1 ===

Sequence: ATGCGTTGGAGAGARCGTTTC; Type: variable 5'/3'; Length: 21; Trimmed: 62861 times.
62821 times, it overlapped the 5' end of a read
40 times, it overlap

4	1	452.1	0	1


Overview of removed sequences (3' or within)
length	count	expect	max.err	error counts
3	2833	1808.3	0	2833
4	7	452.1	0	7
5	6	113.0	0	6

=== Adapter 2 ===

Sequence: GATCACCTTCTAATTTACCWACAACTG; Type: variable 5'/3'; Length: 27; Trimmed: 2610 times.
2463 times, it overlapped the 5' end of a read
147 times, it overlapped the 3' end or was within the read

No. of allowed errors:
0-20 bp: 0; 21-27 bp: 1

Overview of removed sequences (5')
length	count	expect	max.err	error counts
3	8	1808.3	0	8
5	1	113.0	0	1
26	2	0.0	1	0 2
27	2451	0.0	1	0 2451
28	1	0.0	1	0 1


Overview of removed sequences (3' or within)
length	count	expect	max.err	error counts
3	139	1808.3	0	139
4	2	452.1	0	2
5	6	113.0	0	6

This is cutadapt 1.9.1 with Python 2.7.16
Command line parameters: -e 0.047619047619 -b ATGCGTTGGAGAGARCGTTTC -b GATCACCTTCTAATTTACCWACAACTG -o /code/sequences/2530.R1.fastq.gz.trimmed.fastq.gz /code/sequences/2530.R1.fastq.gz
Trimming 2 adapters with at most 4.8% errors in single-end mo

300	5	0.0	0	5

=== Adapter 2 ===

Sequence: GATCACCTTCTAATTTACCWACAACTG; Type: variable 5'/3'; Length: 27; Trimmed: 3386 times.
65 times, it overlapped the 5' end of a read
3321 times, it overlapped the 3' end or was within the read

No. of allowed errors:
0-20 bp: 0; 21-27 bp: 1

Overview of removed sequences (5')
length	count	expect	max.err	error counts
3	61	1866.5	0	61
27	4	0.0	1	2 2


Overview of removed sequences (3' or within)
length	count	expect	max.err	error counts
3	11	1866.5	0	11
4	3310	466.6	0	3310

This is cutadapt 1.9.1 with Python 2.7.16
Command line parameters: -e 0.047619047619 -b ATGCGTTGGAGAGARCGTTTC -b GATCACCTTCTAATTTACCWACAACTG -o /code/sequences/2873.R1.fastq.gz.trimmed.fastq.gz /code/sequences/2873.R1.fastq.gz
Trimming 2 adapters with at most 4.8% errors in single-end mode ...
Finished in 5.51 s (53 us/read; 1.13 M reads/minute).

=== Summary ===

Total reads processed:                 104,259
Reads with adapters:                    86,109 (82.6%)
Reads written (

2152 times, it overlapped the 5' end of a read
109 times, it overlapped the 3' end or was within the read

No. of allowed errors:
0-20 bp: 0; 21-27 bp: 1

Overview of removed sequences (5')
length	count	expect	max.err	error counts
4	6	347.7	0	6
23	1	0.0	1	0 1
27	2145	0.0	1	0 2145


Overview of removed sequences (3' or within)
length	count	expect	max.err	error counts
3	102	1390.7	0	102
4	4	347.7	0	4
5	2	86.9	0	2
6	1	21.7	0	1

This is cutadapt 1.9.1 with Python 2.7.16
Command line parameters: -e 0.047619047619 -b ATGCGTTGGAGAGARCGTTTC -b GATCACCTTCTAATTTACCWACAACTG -o /code/sequences/3135.R2.fastq.gz.trimmed.fastq.gz /code/sequences/3135.R2.fastq.gz
Trimming 2 adapters with at most 4.8% errors in single-end mode ...
Finished in 5.95 s (41 us/read; 1.47 M reads/minute).

=== Summary ===

Total reads processed:                 145,426
Reads with adapters:                     7,747 (5.3%)
Reads written (passing filters):       145,426 (100.0%)

Total basepairs processed:    42,687,990 bp
To


FastQ paired records kept: 176648 (88324 pairs)
FastQ single records kept: 14210 (from PE1: 12110, from PE2: 2100)
FastQ paired records discarded: 2816 (1408 pairs)
FastQ single records discarded: 14210 (from PE1: 2100, from PE2: 12110)



FastQ paired records kept: 300182 (150091 pairs)
FastQ single records kept: 8456 (from PE1: 6296, from PE2: 2160)
FastQ paired records discarded: 1950 (975 pairs)
FastQ single records discarded: 8456 (from PE1: 2160, from PE2: 6296)



FastQ paired records kept: 150902 (75451 pairs)
FastQ single records kept: 5037 (from PE1: 4286, from PE2: 751)
FastQ paired records discarded: 916 (458 pairs)
FastQ single records discarded: 5037 (from PE1: 751, from PE2: 4286)



FastQ paired records kept: 90788 (45394 pairs)
FastQ single records kept: 2528 (from PE1: 2066, from PE2: 462)
FastQ paired records discarded: 426 (213 pairs)
FastQ single records discarded: 2528 (from PE1: 462, from PE2: 2066)



FastQ paired records kept: 31866 (15933 pairs)
FastQ single 


FastQ paired records kept: 254180 (127090 pairs)
FastQ single records kept: 9249 (from PE1: 7482, from PE2: 1767)
FastQ paired records discarded: 1704 (852 pairs)
FastQ single records discarded: 9249 (from PE1: 1767, from PE2: 7482)



FastQ paired records kept: 243418 (121709 pairs)
FastQ single records kept: 3339 (from PE1: 2898, from PE2: 441)
FastQ paired records discarded: 426 (213 pairs)
FastQ single records discarded: 3339 (from PE1: 441, from PE2: 2898)



FastQ paired records kept: 279648 (139824 pairs)
FastQ single records kept: 3884 (from PE1: 3295, from PE2: 589)
FastQ paired records discarded: 508 (254 pairs)
FastQ single records discarded: 3884 (from PE1: 589, from PE2: 3295)



FastQ paired records kept: 175182 (87591 pairs)
FastQ single records kept: 12575 (from PE1: 10105, from PE2: 2470)
FastQ paired records discarded: 2778 (1389 pairs)
FastQ single records discarded: 12575 (from PE1: 2470, from PE2: 10105)



FastQ paired records kept: 181838 (90919 pairs)
FastQ sin


FastQ paired records kept: 395418 (197709 pairs)
FastQ single records kept: 13485 (from PE1: 11320, from PE2: 2165)
FastQ paired records discarded: 2424 (1212 pairs)
FastQ single records discarded: 13485 (from PE1: 2165, from PE2: 11320)



FastQ paired records kept: 207568 (103784 pairs)
FastQ single records kept: 2841 (from PE1: 2423, from PE2: 418)
FastQ paired records discarded: 418 (209 pairs)
FastQ single records discarded: 2841 (from PE1: 418, from PE2: 2423)



FastQ paired records kept: 235324 (117662 pairs)
FastQ single records kept: 3570 (from PE1: 2826, from PE2: 744)
FastQ paired records discarded: 616 (308 pairs)
FastQ single records discarded: 3570 (from PE1: 744, from PE2: 2826)



FastQ paired records kept: 170484 (85242 pairs)
FastQ single records kept: 6519 (from PE1: 5418, from PE2: 1101)
FastQ paired records discarded: 1298 (649 pairs)
FastQ single records discarded: 6519 (from PE1: 1101, from PE2: 5418)



FastQ paired records kept: 218236 (109118 pairs)
FastQ si


FastQ paired records kept: 266490 (133245 pairs)
FastQ single records kept: 8203 (from PE1: 6866, from PE2: 1337)
FastQ paired records discarded: 1470 (735 pairs)
FastQ single records discarded: 8203 (from PE1: 1337, from PE2: 6866)



FastQ paired records kept: 174904 (87452 pairs)
FastQ single records kept: 6158 (from PE1: 5064, from PE2: 1094)
FastQ paired records discarded: 1156 (578 pairs)
FastQ single records discarded: 6158 (from PE1: 1094, from PE2: 5064)



FastQ paired records kept: 532664 (266332 pairs)
FastQ single records kept: 19859 (from PE1: 15968, from PE2: 3891)
FastQ paired records discarded: 4106 (2053 pairs)
FastQ single records discarded: 19859 (from PE1: 3891, from PE2: 15968)



FastQ paired records kept: 3724 (1862 pairs)
FastQ single records kept: 285 (from PE1: 204, from PE2: 81)
FastQ paired records discarded: 54 (27 pairs)
FastQ single records discarded: 285 (from PE1: 81, from PE2: 204)



FastQ paired records kept: 251830 (125915 pairs)
FastQ single reco


FastQ paired records kept: 240230 (120115 pairs)
FastQ single records kept: 3107 (from PE1: 2613, from PE2: 494)
FastQ paired records discarded: 452 (226 pairs)
FastQ single records discarded: 3107 (from PE1: 494, from PE2: 2613)



FastQ paired records kept: 480336 (240168 pairs)
FastQ single records kept: 10148 (from PE1: 7849, from PE2: 2299)
FastQ paired records discarded: 1906 (953 pairs)
FastQ single records discarded: 10148 (from PE1: 2299, from PE2: 7849)



FastQ paired records kept: 9214 (4607 pairs)
FastQ single records kept: 1297 (from PE1: 883, from PE2: 414)
FastQ paired records discarded: 380 (190 pairs)
FastQ single records discarded: 1297 (from PE1: 414, from PE2: 883)




### 3. Generate sequence counts file used in final step to produce reports

In [ ]:
%%time
!for file in sequences/*.passedQC.fastq; \
do \
  awk 'NR%4==2{sum+=1}END{print FILENAME,sum}' $file >> sequences/diatomSequenceCounts.txt; \
done

### 4. Assign similar sequences to OTUs using user-defined similarity threshold

In [ ]:
%%time
!pick_otus.py -i sequences/readyForQiime.allsamples.fasta -o sequences/picked_otus_97

###  5. Pick a representative set of sequences. For each OTU, one sequence will be used in subsequent analysis

In [ ]:
%%time
!pick_rep_set.py -i sequences/picked_otus_97/readyForQiime.allsamples_otus.txt \
  -f sequences/readyForQiime.allsamples.fasta \
  -o sequences/repset.fasta

### 6. Query BLAST database with OTU representatives

In [ ]:
%%time
!blastn -db diatoms -query sequences/repset.fasta \
  -out sequences/repset.diatoms.blastn \
  -task blastn -max_target_seqs 1 -num_threads 8 -outfmt 6 -evalue 0.01

In [ ]:
!mkdir sequences/assigned_taxonomy

### 7. Create taxonomy assignments from BLAST outputs

In [ ]:
%%time
!python ./create_taxonomy_assignments_from_blast.py --taxonomy diatoms.taxonomy.FINAL2017.txt \
  --percid 95.0 --blast sequences/repset.diatoms.blastn --output sequences/assigned_taxonomy/repset.taxonomy.txt 

### 8. Reports how often an OTU is found in each sample and adds the taxonomic predictions for each OTU

In [ ]:
%%time
!make_otu_table.py -i sequences/picked_otus_97/readyForQiime.allsamples_otus.txt \
  -t sequences/assigned_taxonomy/repset.taxonomy.txt \
  -o sequences/otu_table.biom

### 9. Filters an OTU table based on taxonomic metadata excluding specific taxa

In [ ]:
%%time
!filter_taxa_from_otu_table.py -i sequences/otu_table.biom \
  -o sequences/otu_table.diatomsonly.biom \
  -n MARINE,NOT_DIATOM,Yellow_green_Algae,None

### 10. Sort OTU table by sample id

In [ ]:
%%time
!sort_otu_table.py -i sequences/otu_table.diatomsonly.biom \
  -o sequences/otu_table.diatomsonly.biom

### 11. Summary information of the representation of taxonomic groups within each sample

In [ ]:
%%time
!summarize_taxa.py -L 1 \
  -i sequences/otu_table.diatomsonly.biom \
  -o sequences/visualised_taxonomy -a

### 12. Produce Diatom reports

In [ ]:
%%time
!python ./produceDiatomReports.py --folder sequences --lookup lookuptable.txt

### 13. Inspect producted Diatom reports

In [ ]:
import pandas as pd

In [ ]:
pd.read_csv('sequences/Abundances.fail.csv')

In [ ]:
pd.read_csv('sequences/Abundances.pass.csv')